## **Lectura 3: Ejercicio en clase**

## Problema de asignación de recursos con una restricción presupuestaria

Supongamos ahora que existe un coste fijo $C_{r,j}$ asociado a la asignación de un recurso $r \in R$ a un trabajo $j \in J$. Supongamos también que existe un presupuesto limitado $B$ que puede utilizarse para asignar trabajos.

El coste de asignar a Carlos, Joe o Mónika a cualquiera de los trabajos es de $\$1.000$ , $\$2.000$ y $\$3.000$ respectivamente. El presupuesto disponible es de $\$5,000$.

### Data

The list $R$ contains the names of the three resources: Carlos, Joe, and Monika.
The list $J$ contains the names of the job positions: Tester, Java Developer, and Architect.

The Gurobi Python ``multidict`` function initialize two dictionaries:
* "scores" defines the matching scores for each resource and job combination.
* "costs" defines the fixed cost associated of assigning a resource to a job.



In [ ]:
# Resource and job sets
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

# Matching score data
# Cost is given in thousands of dollars
combinations, scores, costs = gp.multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# Available budget (thousands of dollars)
budget = 5

The following constructor creates an empty ``Model`` object “m”. The ``Model`` object “m” holds a single optimization problem. It consists of a set of variables, a set of constraints, and the objective function.

In [ ]:
# Declare and initialize model
m = gp.Model('RAP2')

### Decision variables

The decision variable $x_{r,j}$ is 1 if $r \in R$ is assigned to job $j \in J$, and 0 otherwise.

The ``Model.addVars()`` method defines the decision variables for the model object “m”.  

Because there is a budget constraint, it is possible that not all of the jobs will be filled. To account for this, we define a new decision variable that indicates whether or not a job is filled.

Let $g_{j}$ be equal 1 if job $j \in J$ is not filled, and 0 otherwise. This variable is a gap variable that indicates that a job cannot be filled.

***Remark:*** For the previous formulation of the RAP, we defined the assignment variables as non-negative and continuous which is the default value of the ``vtype`` argument of the ``Model.addVars()`` method.
However, in this extension of the RAP, because of the budget constraint we added to the model, we need to explicitly define these variables as binary. The ``vtype=GRB.BINARY`` argument of the ``Model.addVars()`` method defines the assignment variables as binary.

In [ ]:
# Create decision variables for the RAP model
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# Create gap variables for the RAP model
g = m.addVars(J, name="gap")

### Job constraints

Since we have a limited budget to assign resources to jobs, it is possible that not all the jobs can be filled. For the job constraints, there are two possibilities either a resource is assigned to fill the job, or this job cannot be filled and we need to declare a gap. This latter possibility is captured by the decision variable $g_j$. Therefore, the job constraints are written as follows.

For each job $j \in J$, exactly one resource must be assigned to the job, or the corresponding $g_j$ variable must be set to 1:

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1.
$$


In [ ]:
# Create job constraints
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), name='job')

### Resource constraints

The constraints for the resources need to ensure that at most one job is assigned to each resource. That is, it is possible that not all the resources are assigned. Therefore, the resource constraints are written as follows.

For each resource $r \in R$, at most one job can be assigned to the resource:

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1.
$$

In [ ]:
# Create resource constraints
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), name='resource')

### Budget constraint

This constraint ensures that the cost of assigning resources to fill job requirements do not exceed the budget available. The costs of assignment and budget are in thousands of dollars.

The cost of filling the Tester job is $1x_{1,1}$, if resource Carlos is assigned, or $2x_{2,1}$, if resource Joe is assigned, or $3x_{3,1}$, if resource Monika is assigned.
Consequently, the cost of filling the Tester job is as follows, where at most one term in this summation will be nonzero.

$$
1x_{1,1} + 2x_{2,1} + 3x_{3,1}.
$$

Similarly, the cost of filling the Java Developer and Architect jobs are defined as follows. The cost of filling the Java Developer job is:

$$
1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}.
$$

The cost of filling the Architect job is:

$$
1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}.
$$

Hence, the total cost of filling the jobs should be less or equal than the budget available.

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) \; +
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) \; +
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) \leq 5
\end{equation}

Each term in parenthesis in the budget constraint can be expressed as follows.

\begin{equation}
(1x_{1,1} + 2x_{2,1} + 3x_{3,1}) = \sum_{r \in R} C_{r,1}x_{r,1}.
\end{equation}

\begin{equation}
(1x_{1, 2} + 2x_{2, 2} + 3x_{3, 2}) = \sum_{r \in R} C_{r,2}x_{r,2}.
\end{equation}

\begin{equation}
(1x_{1, 3} + 2x_{2, 3} + 3x_{3, 3}) = \sum_{r \in R} C_{r,3}x_{r,3}.
\end{equation}

Therefore, the budget constraint can be concisely written as:

\begin{equation}
\sum_{j \in J} \sum_{r \in R} C_{r,j}x_{r,j} \leq B.
\end{equation}

The ``Model.addConstr()`` method of the Gurobi/Python API defines the budget constraint of the ``Model`` object “m”.
The first argument of this method, "x.prod(costs)", is the prod method and defines the LHS of the budget constraint. The $<=$ defines a less or equal constraint, and the budget amount available is the RHS of the constraint.
This constraint is saying that the total cost of assigning resources to fill jobs requirements cannot exceed the budget available.
The second argument is the name of this constraint.

In [ ]:
budget = m.addConstr((x.prod(costs) <= budget), name='budget')

## Objective function

The objective function is similar to the RAP. The first term in the objective is the total matching score of the assignments. In this extension of the RAP, it is possible that not all jobs are filled; however, we want to heavily penalize this possibility. For this purpose, we have a second term in the objective function that takes the summation of the gap variables over all the jobs and multiply it by a big penalty $M$.

Observe that the maximum value of a matching score is 100, and the value that we give to $M$ is 101. The rationale behind the value of $M$ is that having gaps heavily deteriorates the total matching scores value.

Consequently, the objective function is to maximize the total matching score of the assignments minus the penalty associated of having gap variables with a value equal to 1.

$$
\max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} s_{r,j}x_{r,j} -M \sum_{j \in J} g_{j}
$$

In [ ]:
# Penalty for not filling a job position
M = 101

In [ ]:
# Objective: maximize total matching score of assignments
# Unfilled jobs are heavily penalized
m.setObjective(x.prod(scores) - M*g.sum(), GRB.MAXIMIZE)

In [ ]:
# Run optimization engine
m.optimize()

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160%     -    0s
     0     0  121.66667    0    7   52.00000  121.66667   134%     -    0s
     0     0     cutoff    0        52.00000   52.00000  0.00%     -    0s

Cutting planes:
  Gomory: 1
  GUB cover: 1
  RLT: 1

Explored 1 nodes (7 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 52 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.200000000000e+01, best bound 5.200000000000e+01, gap 0.0000%


The definition of the objective function includes the penalty of no filling jobs. However, we are interested in the optimal total matching score value when not all the jobs are filled. For this purpose, we need to compute the total matching score value using the matching score values $s_{r,j}$ and the assignment decision variables $x_{r,j}$.

In [ ]:
# Compute total matching score from assignment variables
total_matching_score = 0
for r, j in combinations:
    if x[r, j].x > 1e-6:
        print(x[r, j].varName, x[r, j].x)
        total_matching_score += scores[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)

assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
Total matching score:  153.0


### Analysis

Recall that the budget is $\$5,000$, and the total  cost associated of allocating the three resources is $\$6,000$. This means that there is not enough budget to allocate the three resources we have. Consequently, the Gurobi Optimizer must choose two resources to fill the jobs demand, leave one job unfilled, and maximize the total matching scores. Notice that the two top matching scores are 80% (Joe for the Tester job) and 73% (Monika for the Java Developer job). Also, notice that the lowest score is 13% (Carlos for the Architect job). Assigning Joe to the Tester job, Monika to the Java Developer job, and nobody to the Architect job costs $\$5,000$  and yields a total matching score of 153. This is the optimal solution found by the Gurobi Optimizer.